Depositos para enviar, orderm de preferencia:
musical_matriz
musical_filal
em seguida onde houver mais estoque


onde days_available = 0 criar observação onde informa que há muito tempo sem estoque

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN_MCENTER = os.getenv("ACCESS_TOKEN_MCENTER")
ACCESS_TOKEN_BUENOSHOPS = os.getenv("ACCESS_TOKEN_BUENOSHOPS")
ACCESS_TOKEN_MUSICALCRIS = os.getenv("ACCESS_TOKEN_MUSICALCRIS")

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

In [2]:
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

### Período a consultar

In [3]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 1).date()
data_fim = datetime(2023, 12, 13).date()
# data_fim = data_fim + timedelta(days=1)  # + 1 dia para pegar a data atual no DB
# data_fim = data_fim - timedelta(days=1)  # + 1 dia para pegar a data atual no DB
print("data_inicio: ", data_inicio)
print("data_fim: ", data_fim)

data_inicio:  2023-11-01
data_fim:  2023-12-13


### Historico de estoque fulfillment

In [4]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim};'"
    # sql_query = f"SELECT * FROM fulfillment_stock WHERE created_at BETWEEN '2023-12-04' AND '2023-12-05';"
    print(sql_query)
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

SELECT * FROM mcenter_fulfillment_stock WHERE created_at BETWEEN '2023-11-01' AND '2023-12-13;'


C:\Users\couti\AppData\Local\Temp\ipykernel_15164\1935300295.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [5]:
# df_stock
# print(df_stock.shape)
# duplicates = df_stock[df_stock.duplicated(keep=False)]
# duplicates

In [6]:
# # datas consultadas, dias em que um produto pode ou não estar disponível
# # df_stock["created_at"].value_counts().index.to_list()
# df_stock["created_at"].value_counts().sort_index()
# df_stock["created_at"].dt.date.value_counts().sort_index()
# # df_stock["created_at"].dt.date.value_counts()

# # len(df_stock["created_at"].dt.date.value_counts().sort_index())

In [7]:
# df_stock

In [8]:
# # buscando anuncio e verificando dias consultados
# result = df_stock[df_stock['ml_inventory_id'] == 'FBAY80623']
# result = result.sort_values(by='created_at')
# result

In [9]:
# # encontrando anuncios pela data
# df_stock["created_at"] = pd.to_datetime(df_stock["created_at"])
# result = df_stock.loc[df_stock['created_at'].dt.date == pd.to_datetime('2023-12-7').date()]
# result

In [10]:
# df_stock[df_stock['created_at']=='2023-12-07 03:01:03.036386']

In [11]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by="created_at", ascending=False)
df_stock["data"] = df_stock["created_at"].dt.date
df_stock = df_stock.drop(["created_at"], axis=1)

In [12]:
# df_stock

In [13]:
# df_stock.dtypes

In [14]:
# df_stock['data'] = pd.to_datetime(df_stock['data'])

# dfx = condf(df_stock, 'data', '2023-11-1')
# dfx

In [15]:
# # A linha abaixo conta as ocorrências de cada valor em 'ml_inventory_id'
# # value_counts = result['ml_inventory_id'].value_counts()
# value_counts = df_stock['ml_inventory_id'].value_counts()

# # Filtra para incluir apenas aqueles que ocorrem mais de 2 vezes
# filtered_counts = value_counts[value_counts > 1]

# filtered_counts.shape
# # filtered_counts
# value_counts

In [16]:
# df_stock[df_stock["detail_status"] == "transfer"]

In [17]:
# condf(df_stock,'ml_inventory_id','CRMK90073')

In [18]:
# condf(df_stock, "detail_status", "transfer")

In [19]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by="data", ascending=False).reset_index(drop=True)

In [20]:
# df_stock

In [21]:
# condf(df_stock, "ml_inventory_id", "DSGP06967")

In [22]:
# print(df_stock.shape)
# df_stock = df_stock.drop_duplicates()
# print(df_stock.shape)

#### Dias em que produto esteve disponível

In [23]:
# condf(df_stock,'ml_inventory_id','ABCB20467')

In [24]:
## Contando dias em que produto esteve disponível
days_available = df_stock.groupby(["ml_inventory_id"])["has_stock"].sum().reset_index()
days_available = days_available.rename(columns={"has_stock": "days_available"})

In [25]:
# days_available

In [26]:
# days_available.sort_values(by='days_available',ascending=False)
# days_available['days_available'].value_counts()

# ## Contando dias em que produto esteve disponível
# days_available = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available = days_available.rename(columns={"has_stock": "days_available"})
# days_available

In [27]:
# condf(df_stock, "ml_inventory_id", "MENA66694")

In [28]:
# ## Contando dias em que produto esteve disponível
# days_available_ = df_stock.groupby(["ml_inventory_id","references_variation_id"])["has_stock"].sum().reset_index()
# days_available_ = days_available_.rename(columns={"has_stock": "days_available"})
# days_available_

In [29]:
# condf(days_available, "ml_inventory_id", "DSGP06967")
# condf(days_available, "ml_inventory_id", "DSGP06967")

In [30]:
# Unindo DFs
df_stock_days = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# df_stock_days_left = df_stock.merge(days_available, on=["ml_inventory_id"], how="left")
# df_stock.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# df_stock = df_stock.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# Verificando o formato após a correção
# print(df_stock.shape)
#

In [31]:
# print(df_stock.shape)
# print(df_stock_days.shape)
# print(df_stock_days_left.shape)
# df_stock_days.sample(3)

In [32]:
# # Unindo DFs
# df_stock_ = df_stock.merge(days_available, on=["ml_inventory_id"], how="inner")
# # df_stock_.rename(columns={'days_available_x': 'days_available'}, inplace=True)  # Corrigindo a sintaxe
# # df_stock_ = df_stock_.drop(columns='days_available_y', axis=1)  # Corrigindo a sintaxe

# # Verificando o formato após a correção
# print(df_stock_.shape)
# df_stock_

In [42]:
# data de hoje
data_de_hoje = datetime.now().date()
data_de_hoje = data_de_hoje - timedelta(days=1)
# print(data_de_hoje)

df_stock_days["data"] = pd.to_datetime(df_stock_days["data"])

# Filtra apenas as linhas onde 'data' é igual à data de hoje
df_stock_today = df_stock_days[df_stock_days["data"].dt.date == data_de_hoje]
df_stock_today = df_stock_today.rename(
    columns={"available_quantity": "available_quantity_today"}
)

# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [43]:
df_stock_today

,ml_inventory_id,available_quantity_today,detail_status,detail_quantity,references_id,references_variation_id,data,has_stock,days_available,total_available_quantity


In [ ]:
# Se detail_status = transfer: available_quantity_today = available_quantity_today + detail_quantity
df_stock_today["total_available_quantity"] = df_stock_today.apply(
    lambda row: row["detail_quantity"] + row["available_quantity_today"]
    if row["detail_status"] == "transfer"
    else row["available_quantity_today"],
    axis=1,
)

df_stock_today["total_available_quantity"] = df_stock_today[
    "total_available_quantity"
].astype("int64")

In [ ]:
# df_stock_today.sample(5)

In [ ]:
# condf(df_stock,'ml_inventory_id','FSNB97805')
# condf(df_stock_today,'ml_inventory_id','FSNB97805')

### Buscando hitorico de orders no BD

In [ ]:
# Buscando histórico de vendas na tabela ml_orders_hist para o período definido
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM mcenter_ml_orders WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# # Ordenando orders por data
# df_orders = df_orders.sort_values(by="date_approved", ascending=False)
# df_orders["data"] = df_orders["date_approved"].dt.date
# df_orders = df_orders.drop(["date_closed","date_approved"], axis=1)

# filtros
df_orders = df_orders[df_orders["logistic_type"] == "fulfillment"]
# df_orders = df_orders.drop(columns=['category_id','pack_id','variation_attributes_id','variation_name','variation_value_id', 'data'])
df_orders = df_orders.drop(
    columns=[
        "category_id",
        "pack_id",
        "variation_attributes_id",
        "variation_name",
        "variation_value_id",
    ]
)

# change column
df_orders["variation_id"] = df_orders["variation_id"].replace("nan", "0", regex=True)

# df_orders = df_orders[df_orders["order_status"] == "paid"]
# df_orders = df_orders[df_orders["payment_status"] == "approved"]


# df_orders = df_orders.drop(
#     ["pack_id", "date_approved", "order_status", "payment_status"], axis=1
# )
df_orders.rename(columns={"quantity": "sold_quantity"}, inplace=True)

In [ ]:
print(df_orders.shape)
df_orders = df_orders.drop_duplicates()
print(df_orders.shape)

In [ ]:
# Ordenando orders por data
df_orders = df_orders.sort_values(by="date_approved", ascending=False)
df_orders["data"] = df_orders["date_approved"].dt.date
df_orders = df_orders.drop(["date_closed", "date_approved"], axis=1)

In [ ]:
df_orders.sample(3)

In [ ]:
# df_orders.head()

In [ ]:
# df_orders
# df_orders['pack_id'].value_counts()

# # condf(df_orders,'pack_id','2000005059931419')

In [ ]:
# print(df_orders.shape)
# df_orders.head(3)

In [ ]:
# condf(df_orders, "ml_code", "MLB1992541482")

#### Total de vendas por ml_code e variation_id

In [ ]:
# df_orders.shape

In [ ]:
# df_orders

In [ ]:
# condf(df_orders,'ml_code','MLB3516894673')

In [ ]:
# Total de vendas por ml_code e id de variação
df_orders_quantity = (
    df_orders.groupby(["ml_code", "variation_id"])["sold_quantity"].sum().reset_index()
)

In [ ]:
df_orders_quantity

In [ ]:
print(f"Número de ml_code únicos: {len(df_orders_quantity['ml_code'].unique())}")
print(
    f"Número de variation_id únicos: {len(df_orders_quantity['variation_id'].unique())}"
)

In [ ]:
# condf(df_orders_quantity,'ml_code','MLB2075516288')
# df_orders_quantity['ml_code'].value_counts()

In [ ]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(
    # df_orders, resultado, on=["ml_code", "variation_id"], how="inner"
    df_orders,
    df_orders_quantity,
    on=["ml_code", "variation_id"],
    how="inner",
)
df_total_sales = df_total_sales.rename(
    columns={"sold_quantity_y": "total_sold_quantity"}
)
df_total_sales = df_total_sales.drop(
    columns=["sold_quantity_x", "order_status", "payment_status"]
)

In [ ]:
print(f"Total de vendas = {df_total_sales.shape[0]}")

In [ ]:
# df_orders.shape

In [ ]:
# df_total_sales.columns

In [ ]:
# df_total_sales.sample()

In [ ]:
# # # Exibir todas as linhas e colunas
# # pd.set_option('display.max_rows', None)
# # pd.set_option('display.max_columns', None)

# # Voltar ao normal
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')


# # condf(df_total_sales,'ml_code','MLB3427769549')
# # # x = condf(df_orders,'ml_code','MLB3427769549')

# # # x = x.sort_values(by='data')
# # # x.head(15)
# # x = df_orders[df_orders['ml_code']=='MLB3427769549']
# # x['data'] = pd.to_datetime(x['data'])
# # x[x['data']=='2023-11-22']
# # # x['sold_quantity'].value_counts()

# # # counts = x[x['sold_quantity'] > 1]['sold_quantity'].value_counts()
# # # counts

# # y = x.groupby('data')['sold_quantity'].sum().reset_index()
# # y

# # # # x

In [ ]:
# x.dtypes

In [ ]:
# df_total_sales['data'].value_counts()

In [ ]:
# condf(df_total_sales, "ml_code", "MLB4127982748")
# condf(df_total_sales, "ml_code", "MLB4127982748")

In [ ]:
# df_total_sales = df_total_sales.drop(["sold_quantity_x", "shipping_id", "data"], axis=1)
# df_total_sales = df_total_sales.drop_duplicates()

In [ ]:
# condf(df_total_sales, "ml_code", "MLB1992541482")

Neste ponto temos o total de itens vendidos de um anúncio por período e a quantidade de dias em que um produto esteve disponível.
precisamos juntar esses dados para calcular, para isso trarei as informações de produtos

In [ ]:
# print(df_total_sales.shape)
# df_total_sales.sample()

In [ ]:
# condf(df_total_sales, 'ml_code', 'MLB3461718736')
# condf(df_orders, 'ml_code', 'MLB3461718736')
# # condf(resultado, 'ml_code', 'MLB3427769549')
# # condf(df_stock, 'ml_code', 'MLB3427769549')

In [ ]:
# print(df_stock_today.shape)
# df_stock_today.sample()

#### Buscando Produtos

In [ ]:
# Buscando dados de produtos na tabela tiny_fulfillment
try:
    conn = psycopg2.connect(**db_config)
    sql_query = "SELECT * FROM mcenter_items"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar tiny_fulfillment: {e}")

except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

# df_codes["ml_code"] = df_codes["ml_code"].apply(lambda x: "MLB" + str(x))
df_codes.rename(columns={"inventory_id": "ml_inventory_id"}, inplace=True)
df_codes = df_codes.drop(["created_at", "updated_at"], axis=1)

In [ ]:
df_not_catalogo = df_codes[df_codes["catalog_listing"] == False]
df_catalogo = df_codes[df_codes["catalog_listing"] == True]

In [ ]:
df_not_catalogo.sample()

In [ ]:
df_catalogo.sample()

In [ ]:
df_not_catalogo

In [ ]:
print(df_not_catalogo.shape)
print(df_catalogo.shape)

In [ ]:
df_catalogo

In [ ]:
# df_orders.sample()

In [ ]:
df_total_sales_cat = pd.merge(
    df_catalogo,
    df_total_sales,
    left_on=["ml_code"],
    right_on=["ml_code"],
    how="left",
)
df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_not_cat = pd.merge(
    df_not_catalogo,
    df_total_sales,
    left_on=["ml_code", "variation_id"],
    right_on=["ml_code", "variation_id"],
    how="left",
)
df_total_sales_not_cat = df_total_sales_not_cat.drop_duplicates()

In [ ]:
df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].fillna(0)
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].fillna(0)

df_total_sales_not_cat["total_sold_quantity"] = df_total_sales_not_cat[
    "total_sold_quantity"
].astype("int64")
df_total_sales_cat["total_sold_quantity"] = df_total_sales_cat[
    "total_sold_quantity"
].astype("int64")

In [ ]:
# df_total_sales_cat.sample(5)
# df_total_sales_not_cat['total_sold_quantity'].value_counts()
# df_total_sales_cat['total_sold_quantity'].value_counts()

In [ ]:
print(df_total_sales_cat.shape)
print(df_total_sales_not_cat.shape)

In [ ]:
# df_total_sales_cat

In [ ]:
# df_total_sales_cat = df_total_sales_cat.drop(
#     columns=["data", "shipping_id", "variation_id_x", "order_id"]
# )
df_total_sales_cat_x = df_total_sales_cat.drop(
    columns=["data", "shipping_id", "variation_id_x", "order_id"]
)
df_total_sales_cat_x = df_total_sales_cat_x.drop_duplicates()
df_total_sales_cat_x.shape

In [ ]:
# df_total_sales_cat_x

In [ ]:
# df_total_sales_cat['value_name'].value_counts()

In [ ]:
# condf(df_total_sales_cat, 'ml_code','MLB3427769549')

In [ ]:
# df_vendas_total_sales_not_cat

In [ ]:
print(df_total_sales_not_cat.shape)

In [ ]:
print(df_total_sales_not_cat.shape)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop(
    columns=["data", "shipping_id", "order_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat_x.drop_duplicates()
print(df_total_sales_not_cat_x.shape)

In [ ]:
# df_total_sales_cat_x

In [ ]:
print(df_total_sales_cat_x.shape)
print(df_total_sales_not_cat_x.shape)

In [ ]:
# df_total_sales_cat['ml_inventory_id'].value_counts()

In [ ]:
# Antes de desabilitar
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# Seu código para exibir o DataFrame

# # Depois de desabilitar
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')


# df_total_sales_cat = df_total_sales_cat.drop_duplicates()

df_total_sales_cat_x = df_total_sales_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)
df_total_sales_not_cat_x = df_total_sales_not_cat.drop_duplicates(
    subset=["ml_code", "ml_inventory_id"]
)

In [ ]:
# df_orders.shape

In [ ]:
# condf(df_total_sales_cat,'ml_inventory_id','WTHI04163')
# condf(y,'ml_inventory_id','WTHI04163')

In [ ]:
# condf(df_total_sales_cat,'ml_inventory_id','AWAZ99732')

In [ ]:
df_total_cat = df_total_sales_cat_x.copy()
df_total_not_cat = df_total_sales_not_cat_x.copy()

In [ ]:
df_total_cat.columns

In [ ]:
# Somando total de vendas por inventory_id
df_sum_qt_sold_cat = (
    df_total_cat.groupby("ml_inventory_id")["total_sold_quantity"].sum().reset_index()
)
df_sum_qt_sold_cat = df_sum_qt_sold_cat.rename(
    columns={"total_sold_quantity": "total_sold_catalog"}
)

df_total_cat = pd.merge(
    df_total_cat,
    df_sum_qt_sold_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)

df_total_cat.rename(columns={"variation_id_y": "variation_id_"})


df_sum_qt_sold_not_cat = (
    df_total_not_cat.groupby("ml_inventory_id")["total_sold_quantity"]
    .sum()
    .reset_index()
)
df_sum_qt_sold_not_cat = df_sum_qt_sold_not_cat.rename(
    columns={"total_sold_quantity": "total_sold_not_catalog"}
)
df_total_not_cat = pd.merge(
    df_total_not_cat,
    df_sum_qt_sold_not_cat[["ml_inventory_id", "total_sold_not_catalog"]],
    on="ml_inventory_id",
    how="left",
)

In [ ]:
print(df_total_cat.shape)
print(df_total_not_cat.shape)

In [ ]:
df_total_cat.rename(columns={"variation_id_y": "variation_id"}, inplace=True)
df_total_cat.shape

In [ ]:
# condf(df_total_cat,'ml_inventory_id','WTHI04163')

In [ ]:
df_total_cat = df_total_cat.drop_duplicates(subset=["ml_inventory_id"])

# df_total_cat[df_total_cat['ml_inventory_id']=='WTHI04163']

pergar ml_inventory_id e sum_total e acrescesntar essas informações no df de não catalogo pelo ml_inventory_id
ao fim deixar somatório do catalogo e não catalogo


In [ ]:
# cols=['ml_inventory_id','total_sold_catalog']
# df_total_cat = df_total_cat[cols]
# df_total_cat.sample()

In [ ]:
df_combined = pd.merge(
    df_total_not_cat,
    df_total_cat[["ml_inventory_id", "total_sold_catalog"]],
    on="ml_inventory_id",
    how="left",
)
# df_combined.sample()

In [ ]:
# df_total_not_cat.shape

In [ ]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [ ]:
# condf(df_combined,'ml_inventory_id','UIUW89800')

In [ ]:
# df_combined['ml_inventory_id'].value_counts()

In [ ]:
print(df_combined.shape)

In [ ]:
df = pd.merge(df_combined, df_stock_today, on="ml_inventory_id", how="left")
df["total_sold_catalog"] = df["total_sold_catalog"].fillna(0).astype("int64")

In [ ]:
df.shape

In [ ]:
days = 30

df["total_sold"] = df["total_sold_catalog"] + df["total_sold_not_catalog"]
# qtd de produtos a enviar no período, caso seja valor negativo produto está acima do esperado para envio(sobrando)
df["period_send_fulfillment"] = np.ceil(
    (df["total_sold"] / df["days_available"]) * days - df["total_available_quantity"]
)

df["period_send_fulfillment"] = df["period_send_fulfillment"].fillna(0)

In [ ]:
def calculate_percentual_send(row):
    if row["days_available"] != 0:
        if ((row["total_sold"] / row["days_available"]) * days) * 0.7 > row["total_available_quantity"]:
            return (row["total_sold"] / row["days_available"]) * days - row["total_available_quantity"]
    return 0

# Aplicando a função à coluna "percentual_send"
df["stock_replenishment"] = df.apply(calculate_percentual_send, axis=1)


In [ ]:
df.sample(5)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df_have_itens = df[df["days_available"] > 0]
df_have_itens.shape

In [ ]:
df_sold_zero = df_have_itens[df_have_itens["total_sold"] == 0]
df_sold = df_have_itens[df_have_itens["total_sold"] > 0]

In [ ]:
# produtos sem estoque no período
df_no_itens = df[df["days_available"] <= 0]
df_no_itens = df_no_itens.drop(columns=["period_send_fulfillment"])
df_no_itens.shape

In [ ]:
print(df_no_itens.shape)
print(df_sold.shape)
print(df_sold_zero.shape)

In [ ]:
df_sold_zero.sample(3)

In [ ]:
df_sold.sample(3)

In [ ]:
dfx = df_have_itens.copy()

In [ ]:
df_sold_zero.columns

In [ ]:
dfx.columns

In [ ]:
cols = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "total_available_quantity",
    "days_available",
    "total_sold_not_catalog",
    "total_sold_catalog",
    "total_sold",
    "period_send_fulfillment",
]

In [ ]:
col = [
    "ml_code",
    "seller_sku",
    "ml_inventory_id",
    "value_name",
    "status",
    "title",
    "total_sold_catalog",
    "available_quantity_today",
    "detail_status",
    "detail_quantity",
    "days_available",
    "total_available_quantity",
    "total_sold_not_catalog",
    "total_sold",
]

df_no_itens = df_no_itens[col]
df_no_itens

In [ ]:
def rename_columns(df):
    return df.rename(
        columns={
            "detail_status": "transfer_status",
            "detail_quantity": "transfer_quantity",
        }
    )


df_sold_zero = rename_columns(df_sold_zero)
df_sold = rename_columns(df_sold)
df_no_itens = rename_columns(df_no_itens)

In [ ]:
df_no_itens.columns

In [ ]:
dfx = dfx.rename(
    columns={"detail_status": "transfer_status", "detail_quantity": "transfer_quantity"}
)

In [ ]:
dfx.sample()
dfx.shape

In [ ]:
dfx

## Acabou!